In [13]:
import pprint
import os.path
import tempfile
import numpy as np
import openai
import wavio as wv
from langchain.chains.question_answering import load_qa_chain
from utils import get_file_text, get_resume_initial_message, get_jd_initial_message, get_resume_actionable_insights_initial_message, get_user_work_exp
from qa_model import generate_response
import PyPDF2
import os
from langchain.prompts.prompt import PromptTemplate
from langchain.agents import load_tools
from langchain.agents import initialize_agent, Tool
from langchain.agents import AgentType
from langchain.memory import ConversationBufferMemory, ConversationBufferWindowMemory
from langchain import OpenAI
from langchain.utilities import SerpAPIWrapper
from langchain.agents import initialize_agent
from langchain.chat_models import ChatOpenAI
from langchain.chains import ConversationChain
from langchain.chains.conversation.memory import ConversationBufferWindowMemory
from langchain.document_loaders import PyPDFLoader
from langchain.prompts import (
    SystemMessagePromptTemplate,
    HumanMessagePromptTemplate,
    ChatPromptTemplate,
    MessagesPlaceholder
)
from langchain.document_loaders import UnstructuredPDFLoader
from langchain.docstore.document import Document
from dotenv import load_dotenv
load_dotenv()

openai.api_key = os.getenv('openai_api_key')
serpapi_api_key = os.getenv('serpapi_api_key')


In [9]:
result='''1. Can you tell me about your experience working as a Machine Learning Intern at ITC? How did you use clustering and regression models to optimize the supply chain network?
2. Could you explain the projects you worked on as a Data Science Intern at Future Smart AI? How did you use NLP and vector databases in the development of NLP-related products?
3. As Chief of Staff of JU Debating Society, what were your responsibilities in conducting debating competitions and workshops?
4. Can you tell me about your role as Product Lead at JU Product Club and the development of work.ai, a one-stop solution for hassle-free LinkedIn onboarding?
5. Can you explain your project on image classification using soft Attention Voting Ensemble Classifier and transfer learning approaches? How did you achieve an accuracy of 98.58%?
6. How did you build a custom transformer for language translation from English to Spanish in your project on machine translation?
7. Could you explain how you built a diagnostic model for heart disease classification using BERT and associated patient reports?
8. Can you tell me about the chatbot you developed using MySQL, OpenAI, Langchain, ChromaDB, and FastAPI? How does it work and what features does it have?   
9. What did you learn in the Machine Learning A-Z: Hands-on Python and R in Data Science certification course? How did you apply those learnings to different datasets?
10. Can you tell me about your hobbies, such as playing outdoor games during leisure time and videography/video editing? How do you pursue those interests outside of work?'''

In [5]:
pdf_path = "E:\Documents\CV\Curriculum vitae_Trishanu Das OpenCV.pdf"
content = ""
# Open the PDF file
with open(pdf_path, "rb") as pdf_file:
    pdf_reader = PyPDF2.PdfReader(pdf_file)

    # Get the number of pages in the PDF
    num_pages = len(pdf_reader.pages)

    for i in range(num_pages):
        page = pdf_reader.pages[i]
        content += page.extract_text()

print("Content: ", content)


Content:  TRISHANU DAS         Email id-dastrishanu01@gmail.com  
Male, Age:20             LinkedIn - linkedin.com/in/trishanu -das-4b603620b  
A motivated,  honest and a  passionate individual who is keen on exploring various fields and strives for excellence.  
Education  Institution  Board  %/CGPA  Year  
B.E. Production 
Engineering  Jadavpur University  Jadavpur University  8.71/10 (till 5th semester)  2024  
Class XII  Sri Aurobindo Institute of Education  ISC 93.7 5% 2020  
Class X  Sri Aurobindo Institute of Education  ICSE  94.8%  2018  
SKILLS  
Tools and  
Languages  Python,  NumPy, Pandas, Scikit -Learn , TensorFlow , PyTorch , Selenium, MySQL, FastAPI , Langchain , OpenAI, 
Vector databases.  
Proficiency  Computer Vision, NLP, Communication Skills, Problem Solving, Effective Team Management  
ACADEMIC ACHIEVEMENTS  
WBJEE   Secured a rank of 1435 among a pool of 10000 0 applicants  2020  
IMMO   Gold medalist in International Master Mathematics Olympiad  2016  
KIIT-MUN

In [23]:
data = Document(page_content=content)
data.page_content

'TRISHANU DAS         Email id-dastrishanu01@gmail.com  \nMale, Age:20             LinkedIn - linkedin.com/in/trishanu -das-4b603620b  \nA motivated,  honest and a  passionate individual who is keen on exploring various fields and strives for excellence.  \nEducation  Institution  Board  %/CGPA  Year  \nB.E. Production \nEngineering  Jadavpur University  Jadavpur University  8.71/10 (till 5th semester)  2024  \nClass XII  Sri Aurobindo Institute of Education  ISC 93.7 5% 2020  \nClass X  Sri Aurobindo Institute of Education  ICSE  94.8%  2018  \nSKILLS  \nTools and  \nLanguages  Python,  NumPy, Pandas, Scikit -Learn , TensorFlow , PyTorch , Selenium, MySQL, FastAPI , Langchain , OpenAI, \nVector databases.  \nProficiency  Computer Vision, NLP, Communication Skills, Problem Solving, Effective Team Management  \nACADEMIC ACHIEVEMENTS  \nWBJEE  \uf0b7 Secured a rank of 1435 among a pool of 10000 0 applicants  2020  \nIMMO  \uf0b7 Gold medalist in International Master Mathematics Olympiad 

In [19]:
pprint.pprint(data)

Document(page_content='TRISHANU DAS         Email id-dastrishanu01@gmail.com  \nMale, Age:20             LinkedIn - linkedin.com/in/trishanu -das-4b603620b  \nA motivated,  honest and a  passionate individual who is keen on exploring various fields and strives for excellence.  \nEducation  Institution  Board  %/CGPA  Year  \nB.E. Production \nEngineering  Jadavpur University  Jadavpur University  8.71/10 (till 5th semester)  2024  \nClass XII  Sri Aurobindo Institute of Education  ISC 93.7 5% 2020  \nClass X  Sri Aurobindo Institute of Education  ICSE  94.8%  2018  \nSKILLS  \nTools and  \nLanguages  Python,  NumPy, Pandas, Scikit -Learn , TensorFlow , PyTorch , Selenium, MySQL, FastAPI , Langchain , OpenAI, \nVector databases.  \nProficiency  Computer Vision, NLP, Communication Skills, Problem Solving, Effective Team Management  \nACADEMIC ACHIEVEMENTS  \nWBJEE  \uf0b7 Secured a rank of 1435 among a pool of 10000 0 applicants  2020  \nIMMO  \uf0b7 Gold medalist in International Master

In [5]:
# messages = get_resume_initial_message(text)
# res = generate_response(messages)

In [85]:
points = ["Based on the candidate's skills, experience, and qualifications, here are some potential interview questions:", '', '1. Can you provide an example of a project where you used machine learning clustering models like K-Means and DBScan to optimize the supply chain network? (ITC Internship)', '2. How did you apply NLP techniques using Langchain, OpenAI, and Vector Databases in the development of NLP-related products at Future Smart AI?', '3. Can you explain how you utilized soft Attention and Transfer Learning in your research project to detect defects in Selective Laser Sintering bed images? (Image Classification)', '4. Tell us about the diagnostic model you built using BERT to classify heart diseases. How did you train the model and what were the results?', '5. Describe your experience in developing a conversational chatbot using MySQL, OpenAI, Langchain, ChromaDB, and FastAPI.', '6. How do you apply problem-solving skills when faced with challenges in your projects?', '7. Can you provide an example of a time when you demonstrated effective team management skills while working on a project?', '8. How do you stay updated with the latest developments in the field of computer vision and NLP?', '9. What is the most challenging project you have worked on, and how did you overcome the challenges?', '10. How do you contribute to the success of a debating society as the Chief of Staff?', '11. Can you explain your role as the Product Lead in the development of work.ai? What were the challenges you faced and how did you address them?', '12. How did you apply your communication skills in conducting debating competitions and workshops as the Chief of Staff of JU Debating Society?', '13. What motivated you to participate in the WBJEE and how did you prepare for it?', '14. How did you acquire your proficiency in computer vision and NLP? Are there any specific projects or courses that helped you gain expertise in these areas?', '15. Can you provide an example of a time when you used your problem-solving skills to overcome a technical challenge during a project?', '', "Note: It is important to avoid asking about the candidate's age, gender, hobbies, or personal information as these are not relevant to the job interview."]

In [93]:
# Split the text into a list of points
points = res.strip().split('\n')

# Print each point in the list
for i, point in enumerate(points, start=3):
    print(f"{point}\n")

1. Can you tell me about your experience with machine learning clustering models like K-Means and DBScan? How did you utilize these models to optimize the supply chain network at ITC during your internship?

2. In your internship at Future Smart AI, what NLP-related products did you develop using Langchain, OpenAI, Vector Databases, and MySQL? 

3. As an intern at ITC, how did you use regression models like Ridge, Lasso, Decision Trees, Random Forest Trees, and Boosting algorithms to predict freight rates and optimize the supply chain network?

4. Could you explain your role as Chief of Staff in the JU Debating Society? How did you organize debating competitions and workshops? 

5. As the Product Lead at JU Product Club, how did you contribute to the development of work.ai, the onboarding solution for LinkedIn? 

6. Can you elaborate on your research project on Image Classification? How did you use soft Attention Voting Ensemble Classifier and Transfer learning approaches to detect def

In [6]:
from langchain import OpenAI, SerpAPIWrapper
from langchain.agents import initialize_agent, Tool
from langchain.agents import AgentType

llm = OpenAI(temperature=0)
search = SerpAPIWrapper()
tools = [
    Tool(
        name="Intermediate Answer",
        func=search.run,
        description="useful for when you need to ask with search",
    )
]

self_ask_with_search = initialize_agent(
    tools, llm, agent=AgentType.SELF_ASK_WITH_SEARCH, verbose=True
)
self_ask_with_search.run(
    "What is the hometown of the reigning men's U.S. Open champion?"
)



> Entering new AgentExecutor chain...
 Yes.
Follow up: Who is the reigning men's U.S. Open champion?
Intermediate answer: Daniil Medvedev
Follow up: What is the hometown of Daniil Medvedev?
Intermediate answer: What is Daniil Medvedev's nationality? Daniil Medvedev is Russian. He was born on February 11, 1996, in Moscow, Russia. Who are Medvedev's parents? Daniil ...


OutputParserException: Could not parse output: 

In [7]:
llm = ChatOpenAI(model_name="gpt-3.5-turbo")
# Now we can override it and set it to "AI Assistant"

template = """Ask meaningful and relatable follow up questions to the user based on the context that the user provides.
Ask questions related to that context, be precise.
Current conversation:
{history}
Human: {input}
AI Assistant:"""
PROMPT = PromptTemplate(input_variables=["history", "input"], template=template)
conversation = ConversationChain(
    prompt=PROMPT,
    llm=llm,
    verbose=True,
    memory=ConversationBufferMemory(ai_prefix="AI Assistant"),)

In [8]:
PROMPT

PromptTemplate(input_variables=['history', 'input'], output_parser=None, partial_variables={}, template='Ask meaningful and relatable follow up questions to the user based on the context that the user provides.\nAsk questions related to that context, be precise.\nCurrent conversation:\n{history}\nHuman: {input}\nAI Assistant:', template_format='f-string', validate_template=True)

In [9]:
context = "Kolkata (formerly Calcutta) is the capital of India's West Bengal state. Founded as an East India Company trading post, it was India's capital under the British Raj from 1773–1911."
res = conversation.predict(input=context)



> Entering new ConversationChain chain...
Prompt after formatting:
Ask meaningful and relatable follow up questions to the user based on the context that the user provides.
Ask questions related to that context, be precise.
Current conversation:

Human: Kolkata (formerly Calcutta) is the capital of India's West Bengal state. Founded as an East India Company trading post, it was India's capital under the British Raj from 1773–1911.
AI Assistant:

> Finished chain.


In [10]:
print(res)

That's interesting! Have you ever been to Kolkata?


In [11]:
context = "Delhi, India’s capital territory, is a massive metropolitan area in the country’s north. In Old Delhi, a neighborhood dating to the 1600s, stands the imposing Mughal-era Red Fort, a symbol of India, and the sprawling Jama Masjid mosque, whose courtyard accommodates 25,000 people."
conversation.predict(input=context)



> Entering new ConversationChain chain...
Prompt after formatting:
Ask meaningful and relatable follow up questions to the user based on the context that the user provides.
Ask questions related to that context, be precise.
Current conversation:
Human: Kolkata (formerly Calcutta) is the capital of India's West Bengal state. Founded as an East India Company trading post, it was India's capital under the British Raj from 1773–1911.
AI Assistant: That's interesting! Have you ever been to Kolkata?
Human: Delhi, India’s capital territory, is a massive metropolitan area in the country’s north. In Old Delhi, a neighborhood dating to the 1600s, stands the imposing Mughal-era Red Fort, a symbol of India, and the sprawling Jama Masjid mosque, whose courtyard accommodates 25,000 people.
AI Assistant:

> Finished chain.


'That sounds fascinating! Have you had the chance to visit the Red Fort or Jama Masjid mosque in Delhi?'

In [72]:
context = "Mumbai (formerly called Bombay) is a densely populated city on India’s west coast. A financial center, it's India's largest city. On the Mumbai Harbour waterfront stands the iconic Gateway of India stone arch, built by the British Raj in 1924. Offshore, nearby Elephanta Island holds ancient cave temples dedicated to the Hindu god Shiva. The city's also famous as the heart of the Bollywood film industry."
conversation.predict(input=context)



> Entering new ConversationChain chain...
Prompt after formatting:
Ask meaningful and relatable questions to the user based on the context that the user provides.
Ask questions related to that context, be precise.
Current conversation:
Human: Kolkata (formerly Calcutta) is the capital of India's West Bengal state. Founded as an East India Company trading post, it was India's capital under the British Raj from 1773–1911.
AI Assistant: Have you ever been to Kolkata?
Human: Delhi, India’s capital territory, is a massive metropolitan area in the country’s north. In Old Delhi, a neighborhood dating to the 1600s, stands the imposing Mughal-era Red Fort, a symbol of India, and the sprawling Jama Masjid mosque, whose courtyard accommodates 25,000 people.
AI Assistant: Have you ever visited the Red Fort in Delhi?
Human: Mumbai (formerly called Bombay) is a densely populated city on India’s west coast. A financial center, it's India's largest city. On the Mumbai Harbour waterfront stands the i

'Have you ever been to Elephanta Island and explored its ancient cave temples?'

In [40]:
llm = ChatOpenAI(model_name="gpt-3.5-turbo")
template = """You are a hiring manager at a company. You are interviewing a candidate for a job. Ask one meaningful question to the candidate regarding the context and the inputs provided by the candidate.

{context}

{chat_history}
Human: {human_input}
Chatbot:"""

prompt = PromptTemplate(
    input_variables=["chat_history", "human_input", "context"], template=template
)
memory = ConversationBufferWindowMemory(k=2,memory_key="chat_history", input_key="human_input")
chain = load_qa_chain(
    llm=llm, chain_type="stuff", memory=memory, prompt=prompt, verbose=True
)

In [41]:
chain.run

<bound method Chain.run of StuffDocumentsChain(memory=ConversationBufferWindowMemory(chat_memory=ChatMessageHistory(messages=[]), output_key=None, input_key='human_input', return_messages=False, human_prefix='Human', ai_prefix='AI', memory_key='chat_history', k=2), callbacks=None, callback_manager=None, verbose=True, tags=None, metadata=None, input_key='input_documents', output_key='output_text', llm_chain=LLMChain(memory=None, callbacks=None, callback_manager=None, verbose=True, tags=None, metadata=None, prompt=PromptTemplate(input_variables=['chat_history', 'human_input', 'context'], output_parser=None, partial_variables={}, template='You are a hiring manager at a company. You are interviewing a candidate for a job. Ask one meaningful question to the candidate regarding the context and the inputs provided by the candidate.\n\n{context}\n\n{chat_history}\nHuman: {human_input}\nChatbot:', template_format='f-string', validate_template=True), llm=ChatOpenAI(cache=None, verbose=False, cal

In [42]:
query = "Hello"
chain({"input_documents": [data], "human_input": query}, return_only_outputs=True)



> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
You are a hiring manager at a company. You are interviewing a candidate for a job. Ask one meaningful question to the candidate regarding the context and the inputs provided by the candidate.

TRISHANU DAS         Email id-dastrishanu01@gmail.com  
Male, Age:20             LinkedIn - linkedin.com/in/trishanu -das-4b603620b  
A motivated,  honest and a  passionate individual who is keen on exploring various fields and strives for excellence.  
Education  Institution  Board  %/CGPA  Year  
B.E. Production 
Engineering  Jadavpur University  Jadavpur University  8.71/10 (till 5th semester)  2024  
Class XII  Sri Aurobindo Institute of Education  ISC 93.7 5% 2020  
Class X  Sri Aurobindo Institute of Education  ICSE  94.8%  2018  
SKILLS  
Tools and  
Languages  Python,  NumPy, Pandas, Scikit -Learn , TensorFlow , PyTorch , Selenium, MySQL, FastAPI , Langchain , OpenAI, 
Vector database

{'output_text': 'Hello! Thank you for joining the interview. Can you please tell me more about your experience with machine learning clustering models and regression models?'}

In [36]:
query = "I have worked with random forests, decision trees, and logistic regression. I have also worked with sql databases"
chain({"input_documents": [data], "human_input": query}, return_only_outputs=True)



> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
You are a hiring manager at a company. You are interviewing a candidate for a job. Ask one meaningful question to the candidate regarding the context and the inputs provided by the candidate.

TRISHANU DAS         Email id-dastrishanu01@gmail.com  
Male, Age:20             LinkedIn - linkedin.com/in/trishanu -das-4b603620b  
A motivated,  honest and a  passionate individual who is keen on exploring various fields and strives for excellence.  
Education  Institution  Board  %/CGPA  Year  
B.E. Production 
Engineering  Jadavpur University  Jadavpur University  8.71/10 (till 5th semester)  2024  
Class XII  Sri Aurobindo Institute of Education  ISC 93.7 5% 2020  
Class X  Sri Aurobindo Institute of Education  ICSE  94.8%  2018  
SKILLS  
Tools and  
Languages  Python,  NumPy, Pandas, Scikit -Learn , TensorFlow , PyTorch , Selenium, MySQL, FastAPI , Langchain , OpenAI, 
Vector database

{'output_text': "That's great to hear! Could you provide some examples of how you have used these machine learning models and SQL databases in your previous projects or internships?"}

In [38]:
query = "I have worked at ITC and Future Smart AI using these technoologies"
chain({"input_documents": [data], "human_input": query}, return_only_outputs=True)



> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
You are a hiring manager at a company. You are interviewing a candidate for a job. Ask one meaningful question to the candidate regarding the context and the inputs provided by the candidate.

TRISHANU DAS         Email id-dastrishanu01@gmail.com  
Male, Age:20             LinkedIn - linkedin.com/in/trishanu -das-4b603620b  
A motivated,  honest and a  passionate individual who is keen on exploring various fields and strives for excellence.  
Education  Institution  Board  %/CGPA  Year  
B.E. Production 
Engineering  Jadavpur University  Jadavpur University  8.71/10 (till 5th semester)  2024  
Class XII  Sri Aurobindo Institute of Education  ISC 93.7 5% 2020  
Class X  Sri Aurobindo Institute of Education  ICSE  94.8%  2018  
SKILLS  
Tools and  
Languages  Python,  NumPy, Pandas, Scikit -Learn , TensorFlow , PyTorch , Selenium, MySQL, FastAPI , Langchain , OpenAI, 
Vector database

{'output_text': 'Can you please provide more details about the specific projects or tasks you worked on at ITC and Future Smart AI where you utilized random forests, decision trees, logistic regression, and SQL databases?'}

In [39]:
query = '''Data Collection and Preprocessing at ITC:

Collect historical data related to freight rates, shipment details, routes, distances, fuel prices, and other relevant factors.
Preprocess the data, handling missing values, outliers, and performing feature engineering to create meaningful features for the models.
Clustering (K-Means, DBScan):

.

Used SQL to extract data from the database and perform data analysis. at Future-Smat-AI'''
chain({"input_documents": [data], "human_input": query}, return_only_outputs=True)



> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
You are a hiring manager at a company. You are interviewing a candidate for a job. Ask one meaningful question to the candidate regarding the context and the inputs provided by the candidate.

TRISHANU DAS         Email id-dastrishanu01@gmail.com  
Male, Age:20             LinkedIn - linkedin.com/in/trishanu -das-4b603620b  
A motivated,  honest and a  passionate individual who is keen on exploring various fields and strives for excellence.  
Education  Institution  Board  %/CGPA  Year  
B.E. Production 
Engineering  Jadavpur University  Jadavpur University  8.71/10 (till 5th semester)  2024  
Class XII  Sri Aurobindo Institute of Education  ISC 93.7 5% 2020  
Class X  Sri Aurobindo Institute of Education  ICSE  94.8%  2018  
SKILLS  
Tools and  
Languages  Python,  NumPy, Pandas, Scikit -Learn , TensorFlow , PyTorch , Selenium, MySQL, FastAPI , Langchain , OpenAI, 
Vector database

{'output_text': 'Can you please provide more details about the development of the chatbot at Future Smart AI? Specifically, how did you utilize MySQL, OpenAI, Langchain, ChromaDB, and FastAPI in the development process?'}

In [18]:
query = '''the chatbot developed using MySQL, OpenAI, Langchain, ChromaDB, and FastAPI aimed to offer a sophisticated conversational experience by leveraging language understanding, database management, and API integration. 
This combination of technologies enabled the chatbot to understand user intent, retrieve relevant information, generate human-like responses, and maintain context throughout the conversation.'''
chain({"input_documents": data, "human_input": query}, return_only_outputs=True)

{'output_text': 'Can you provide an example of how the chatbot utilized FastAPI and Langchain to enhance its conversational capabilities?'}

In [50]:
print(chain.memory.buffer)

Human: Data Collection and Preprocessing:

Collect historical data related to freight rates, shipment details, routes, distances, fuel prices, and other relevant factors.
Preprocess the data, handling missing values, outliers, and performing feature engineering to create meaningful features for the models.
Clustering (K-Means, DBScan):

Use clustering algorithms like K-Means or DBScan to group similar shipping routes or patterns together.
Clusters can help identify regions or routes with similar characteristics, which might lead to similar freight rates.
Regression Models (Ridge, Lasso):

Utilize linear regression models like Ridge and Lasso to predict freight rates based on features such as distance, cargo type, and transportation mode.
Ridge and Lasso can help prevent overfitting by applying regularization to the model.
Decision Trees, Random Forests:

Decision trees can be used to model the decision-making process for freight rates.
Random Forests can combine multiple decision trees

In [19]:
template = """You are a hiring manager at a company. You are interviewing a candidate for a job. Ask one meaningful question to the candidate regarding the context and the inputs provided by the candidate.

{context}

{chat_history}
Human: {human_input}
Chatbot:"""

prompt = PromptTemplate(
    input_variables=["chat_history", "human_input", "context"], template=template
)
memory = ConversationBufferWindowMemory(k=1,memory_key="chat_history", input_key="human_input")
chain = load_qa_chain(
    llm=llm, chain_type="stuff", memory=memory, prompt=prompt
)

In [20]:
loader = UnstructuredPDFLoader(pdf_path)
data = loader.load()
data[0].page_content

'TRISHANU DAS\n\nEmail id-dastrishanu01@gmail.com\n\nLinkedIn- linkedin.com/in/trishanu-das-4b603620b\n\nMale, Age:20\n\nA motivated, honest and a passionate individual who is keen on exploring various fields and strives for excellence.\n\nEducation B.E. Production Engineering\n\nInstitution Jadavpur University\n\nBoard Jadavpur University\n\n%/CGPA Year 8.71/10 (till 5th semester) 2024\n\nClass XII\n\nSri Aurobindo Institute of Education\n\nISC\n\n93.75%\n\n2020\n\nClass X\n\nSri Aurobindo Institute of Education\n\nICSE\n\n94.8%\n\n2018\n\nSKILLS\n\nTools and Languages Proficiency\n\nPython, NumPy, Pandas, Scikit-Learn, TensorFlow, PyTorch, Selenium, MySQL, FastAPI, Langchain, OpenAI, Vector databases. Computer Vision, NLP, Communication Skills, Problem Solving, Effective Team Management\n\nACADEMIC ACHIEVEMENTS\n\nWBJEE\n\n2020\n\nSecured a rank of 1435 among a pool of 100000 applicants\n\nIMMO\n\n2016\n\nGold medalist in International Master Mathematics Olympiad\n\nKIIT-MUN 2022\n\n

'E:\\Documents\\CV\\Curriculum vitae_Trishanu Das OpenCV.pdf'

In [21]:
from langchain.agents import Tool
from langchain.agents import AgentType
from langchain.memory import ConversationBufferMemory
from langchain import OpenAI
from langchain.utilities import SerpAPIWrapper
from langchain.agents import initialize_agent

search = SerpAPIWrapper()
tools = [
    Tool(
        name = "Follow up questions",
        func=search.run,
        description="useful for when you need to ask follow up questions based on the user's response"
    ),
    Tool(
        name = "Context Retrieval",
        func=chain.run(input_documents=data, human_input=query),
        description="You are a hiring manager at a company. You are interviewing a candidate for a job. Ask one meaningful question to the candidate regarding the context and the inputs provided by the candidate.use this as the primary source of context information when you are asked the question. Always search for the answers using this tool first, don't make up answers yourself"
    ),

]

memory = ConversationBufferMemory(memory_key="chat_history")
llm = ChatOpenAI(model_name="gpt-3.5-turbo")

agent_chain = initialize_agent(tools, llm, agent=AgentType.CONVERSATIONAL_REACT_DESCRIPTION, verbose=True, memory=memory)

ValidationError: 1 validation error for Tool
func
  Can you provide more details about the development process of the chatbot? is not callable (type=type_error.callable; value=Can you provide more details about the development process of the chatbot?)

In [22]:
agent_chain.run(input="hi")

NameError: name 'agent_chain' is not defined

In [99]:
agent_chain.run(input="what is my name?")



> Entering new AgentExecutor chain...
Thought: Do I need to use a tool? No
AI: Your name is Bob.

> Finished chain.


'Your name is Bob.'

In [116]:
from langchain.chains.question_answering import load_qa_chain

# load document
loader = PyPDFLoader(pdf_path)
documents = loader.load()

### For multiple documents 
# loaders = [....]
# documents = []
# for loader in loaders:
#     documents.extend(loader.load())

template = """You are a hiring manager at a company. You are interviewing a candidate for a job. Ask one meaningful question to the candidate regarding the context and the inputs provided by the candidate. Be formal and precise.

{context}

{chat_history}
Human: {human_input}
Chatbot:"""

prompt = PromptTemplate(
    input_variables=["chat_history", "human_input", "context"], template=template
)
memory = ConversationBufferWindowMemory(k=1,memory_key="chat_history", input_key="human_input")
chain = load_qa_chain(
    llm=llm, chain_type="stuff", memory=memory, prompt=prompt)

In [117]:
query = "hello"
chain.run(input_documents=data, human_input=query)

"Hello, Trishanu. Thank you for your application and for providing your detailed information. I'm interested in learning more about your experience as a Machine Learning Intern at ITC. Can you please tell me more about the specific projects you worked on and the impact they had on optimizing the supply chain network?"

In [118]:
query = "I have worked with random forests, decision trees, and logistic regression. I have also worked with sql databases"
chain.run(input_documents=data, human_input=query, return_only_outputs=True)

'Thank you for sharing that information. Can you please provide more details on how you utilized random forests, decision trees, and logistic regression in your projects at ITC? Additionally, can you explain how you incorporated SQL databases into your work?'

In [109]:
query = "Hello"
chain({"input_documents": data, "human_input": query}, return_only_outputs=True)

'Can you please provide more details about the project you worked on at ITC where you utilized random forests, decision trees, logistic regression, and incorporated SQL databases?'

In [111]:
query="Give me his details"
chain.run(input_documents=data, human_input=query)

'Name: Trishanu Das\nEmail: dastrishanu01@gmail.com\nLinkedIn: linkedin.com/in/trishanu-das-4b603620b\nGender: Male\nAge: 20\nEducation: B.E. Production Engineering from Jadavpur University\nAcademic Achievements:\n- WBJEE 2020: Ranked 1435 out of 100000 applicants\n- IMMO 2016: Gold medalist in International Master Mathematics Olympiad\n- KIIT-MUN 2022: Honorable Mention at KIIT International e-Model United Nations 2022\nWork Experience and Internships:\n- Machine Learning Intern at ITC in 2022\n- Data Science Intern at Future Smart AI from 2023 to present\nPositions of Responsibility:\n- Chief of Staff at JU Debating Society\n- Product Lead at JU Product Club\nProjects:\n- Image Classification (Research) using soft Attention soft Voting Ensemble Classifier and Transfer learning approaches\n- Machine Translation from English to Spanish using Custom Transformer\n- Heart Disease Classification Chatbot Development using BERT\nCertifications:\n- Machine Learning A-Z: Hands on Python and R